Install Required Packages
Run this in a notebook cell:

In [1]:
!pip install python-docx openai psycopg2 numpy


In [5]:
!pip uninstall -y azure-search-documents


In [7]:
!pip uninstall -y azure-search-documents


In [8]:
!pip show azure-search-documents


In [9]:
!pip install --pre azure-search-documents==11.6.0b11


  Using cached azure_search_documents-11.6.0b11-py3-none-any.whl.metadata (22 kB)
Using cached azure_search_documents-11.6.0b11-py3-none-any.whl (338 kB)


In [3]:
!pip install -r requirements.txt

ERROR: Cannot install azure-search-documents==11.4.0 and azure-search-documents==11.6.0b11 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested azure-search-documents==11.4.0
    The user requested azure-search-documents==11.6.0b11

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
!pip install python-docx


In [3]:
!pip show python-docx


Name: python-docx
Version: 1.1.2
Summary: Create, read, and update Microsoft Word .docx files.
Home-page: 
Author: 
Author-email: Steve Canny <stcanny@gmail.com>
License: MIT
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: lxml, typing-extensions
Required-by: 


In [8]:
!pip install openai==0.28.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.11.1
    Uninstalling openai-1.11.1:
      Successfully uninstalled openai-1.11.1


# Upload and Extract Text from .docx Files
Add the following in a new cell:

In [1]:
import os
from docx import Document


def extract_text_from_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# List of .docx files in the Data folder
docx_files = [
    "Data/Fleet maintenance best practices across industries.docx",
    "Data/Top 10 tips on Fleet Preventive Maintenance.docx",
    "Data/Fleet vehicle maintenance A comprehensive guide.docx",
    "Data/Fleet_Maintenance_Tips.docx"
]

documents = {}

for file_path in docx_files:
    file_name = os.path.basename(file_path)
    text = extract_text_from_docx(file_path)
    documents[file_name] = text


## Generate Embeddings Using Azure OpenAI
Replace the placeholder values with your Azure OpenAI config:
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")

In [ ]:
import openai

# Replace with your actual Azure OpenAI values
openai.api_type = "azure"
openai.api_base = ""  # No trailing slash
openai.api_version = "2024-07-01-preview"  # Use the latest supported version
openai.api_key = ""

# The deployment name of your embedding model in Azure OpenAI (not model name!)
DEPLOYMENT_NAME = "text-embedding-ada-002"

def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        engine=DEPLOYMENT_NAME
    )
    return response['data'][0]['embedding']

# Assuming `documents` is a dict: {filename: content}
embeddings = {
    filename: get_embedding(content)
    for filename, content in documents.items()
}


## Insert Documents & Embeddings into PostgreSQL Table
Now connect to your PostgreSQL server and insert the data:

In [ ]:
import psycopg2
import numpy as np

# Connection details - update as per your PostgreSQL server
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require'  # required for Azure
)

cur = conn.cursor()

# Insert documents into PostgreSQL table
for filename, content in documents.items():
    embedding_vector = np.array(embeddings[filename]).tolist()
    
    cur.execute(
        "INSERT INTO maintenance_documents (filename, content, embeddings) VALUES (%s, %s, %s)",
        (filename, content, embedding_vector)
    )

conn.commit()
cur.close()
conn.close()
print("Documents and embeddings inserted successfully.")


Documents and embeddings inserted successfully.


In [ ]:
import psycopg2
import numpy as np
import json  # Optional: to pretty-print

# Connect to the database
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require' 
)

cur = conn.cursor()

# Fetch records from the table
cur.execute("SELECT filename, content, embeddings FROM maintenance_documents LIMIT 5")  # You can change the LIMIT

rows = cur.fetchall()

for row in rows:
    filename, content, embedding = row
    print(f"Filename: {filename}")
    print(f"Content: {content[:100]}...")  # Print a snippet of content
    print(f"Embedding Vector Length: {len(embedding)}")
    print(f"First 5 Embedding Values: {embedding[:5]}\n")  # Show first few values



cur.close()
conn.close()


Filename: Fleet maintenance best practices across industries.docx
Content: Fleet maintenance best practices across industries.
Fleet maintenance helps keep vehicles in good co...
Embedding Vector Length: 19387
First 5 Embedding Values: [0.01

Filename: Top 10 tips on Fleet Preventive Maintenance.docx
Content: Top 10 tips on Fleet Preventive Maintenance
Discover the top 10 tips on preventive maintenance and l...
Embedding Vector Length: 19394
First 5 Embedding Values: [0.01

Filename: Fleet vehicle maintenance A comprehensive guide.docx
Content: Fleet vehicle maintenance: A comprehensive guide
Fleet vehicle maintenance is essential for keeping ...
Embedding Vector Length: 19392
First 5 Embedding Values: [0.01

Filename: Fleet_Maintenance_Tips.docx
Content: Fleet Maintenance Tips & Cost Estimation Guide
This document provides a detailed guide on fleet main...
Embedding Vector Length: 19411
First 5 Embedding Values: [-0.0

Filename: Fleet maintenance best practices across industries.docx


    dbname="your_dbname",
    user="your_username",
    password="your_password",
    host="your_postgresql_flexible_server_name.postgres.database.azure.com",
    port="5432",
    sslmode='require'

In [ ]:
import psycopg2
import numpy as np

# Connect to the PostgreSQL DB
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require' 
)

cur = conn.cursor()

# Fetch rows (adjust LIMIT as needed)
cur.execute("SELECT filename, content, embeddings FROM maintenance_documents LIMIT 5")
rows = cur.fetchall()

for row in rows:
    filename, content, embedding = row
    print(f"Filename: {filename}")
    print(f"Content Preview: {content[:100]}...")  # Show first 100 characters
    print(f"Embedding (first 10 values): {embedding[:10]}\n")  # Only print 10 embedding values

cur.close()
conn.close()


Filename: Fleet maintenance best practices across industries.docx
Content Preview: Fleet maintenance best practices across industries.
Fleet maintenance helps keep vehicles in good co...
Embedding (first 10 values): [0.0152893

Filename: Top 10 tips on Fleet Preventive Maintenance.docx
Content Preview: Top 10 tips on Fleet Preventive Maintenance
Discover the top 10 tips on preventive maintenance and l...
Embedding (first 10 values): [0.0145332

Filename: Fleet vehicle maintenance A comprehensive guide.docx
Content Preview: Fleet vehicle maintenance: A comprehensive guide
Fleet vehicle maintenance is essential for keeping ...
Embedding (first 10 values): [0.0154493

Filename: Fleet_Maintenance_Tips.docx
Content Preview: Fleet Maintenance Tips & Cost Estimation Guide
This document provides a detailed guide on fleet main...
Embedding (first 10 values): [-0.009083



In [19]:
query_text = "how to reduce fleet repair costs"
query_embedding = get_embedding(query_text)


 Run Semantic Search (Cosine Similarity Query)
In PostgreSQL, use <=> to calculate cosine distance if you're using pgvector.

In [ ]:
import psycopg2
import numpy as np

# Create vector string from Python embedding
vector_str = str(query_embedding).replace('[', '{').replace(']', '}')  # Convert to PostgreSQL vector format

# Connect and execute search
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require' 
)
cur = conn.cursor()

sql = f"""
    SELECT filename, content, embeddings <=> '{vector_str}'::vector AS similarity
    FROM maintenance_documents
    ORDER BY embeddings <=> '{vector_str}'::vector
    LIMIT 5;
"""

cur.execute(sql)
results = cur.fetchall()
for filename, content, similarity in results:
    print(f"Filename: {filename}")
    print(f"Similarity Score: {similarity}")
    print(f"Content Preview: {content[:100]}...\n")

cur.close()
conn.close()


InvalidTextRepresentation: invalid input syntax for type vector: "{0.0017407250124961138, 0.0065658604726195335, -0.005753399338573217, -0.003382190130650997, -0.015616895630955696, 0.00763198547065258, -0.02258714847266674, 0.00840400718152523, -0.01222735270857811, -0.019675523042678833, -0.004474049434065819, 0.027366328984498978, -0.0003637232875917107, 0.02855744957923889, -0.002420839387923479, 0.00938925426453352, 0.016425680369138718, -0.03152789548039436, 0.03752760589122772, -0.015337496995925903, -0.02568993903696537, 0.004801239352673292, -0.0030770576559007168, -0.02164601720869541, 0.006356311496347189, -0.01961670257151127, 0.03396895155310631, -0.010425969026982784, -0.023057714104652405, 0.006382045801728964, 0.024837039411067963, -0.01858733966946602, 0.004010836128145456, 0.0011451654136180878, -0.009977460838854313, 0.012043537572026253, 0.02754279226064682, -0.008962803520262241, 0.009690710343420506, -0.03185140714049339, 0.04143918305635452, -0.002628549933433532

In [ ]:
import psycopg2
import numpy as np

# Assume query_embedding is a list or numpy array
query_embedding = np.array(query_embedding).tolist()

# Connect to the database
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require'
)
cur = conn.cursor()

# SQL to get only the embeddings of top 5 most similar records
sql = """
    SELECT embeddings
    FROM maintenance_documents
    ORDER BY embeddings <=> %s::vector
    LIMIT 5;
"""

cur.execute(sql, (query_embedding,))
results = cur.fetchall()

# Each result is a tuple with one element: the embedding vector
for (embedding,) in results:
    print(embedding)  # Just prints the full embedding vector

cur.close()
conn.close()


[0.014533247,0.0027200652,0.0050203972,-0.028751696,-0.008873413,-0.0027217048,0.0011050777,-0.0050466307,0.0007980661,-0.0057680462,-0.015215313,0.026915364,-0.015884262,0.013575732,-0.0003467714,0.001819935,0.03704142,-0.01314944,0.017851759,-0.013201907,-0.029302593,-0.009922745,-0.026521863,-0.01534648,-0.0014018419,-0.0053253593,0.028620528,-0.021458838,-0.02246882,-0.008938996,0.016290879,-0.0074896067,0.003049621,0.00093374145,-0.008702897,0.002183922,0.026141481,-0.03197839,-0.0108409105,-0.015726862,0.036411818,0.016815543,-0.0038267823,-0.0024757674,-0.00012737497,0.021183388,0.013785598,0.010434294,-0.009791578,0.0073322067,-0.0012124702,0.005351593,-0.014323381,-0.011595118,-0.011490185,0.0020445576,-0.009194771,0.036936484,-0.016999178,-0.023373868,-0.019832373,0.018599408,-0.028909095,0.0004881853,0.001182138,-0.00033713886,0.0023150886,0.017169693,-0.019386407,-0.008932438,0.025774214,0.034890287,0.012821524,0.008381539,0.0054204552,-0.03452302,-0.0024872446,0.003088971,

 ## Export Data from PostgreSQL (for use in AML + Azure AI Search)#

In [ ]:
import psycopg2
import json

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname="flexibleserverdb",
    user="citus",
    password="",
    host="",
    port="5432",
    sslmode='require'
)

cur = conn.cursor()

# Fetch filename, content, and embeddings from the table
cur.execute("""
    SELECT filename, content, embeddings
    FROM maintenance_documents;
""")

records = cur.fetchall()
cur.close()
conn.close()

# Convert to list of documents for Azure AI Search
documents = []
for i, (filename, content, embedding) in enumerate(records):
    documents.append({
        "@search.action": "upload",
        "id": str(i),
        "filename": filename,
        "content": content,
        "embedding": embedding  # Ensure this is a list of floats
    })

# Optional: save as JSON file for inspection or upload
with open("fleet_docs_with_embeddings.json", "w") as f:
    json.dump(documents, f)

print("✅ Export complete. Ready to push to Azure AI Search.")


✅ Export complete. Ready to push to Azure AI Search.


## Create Azure AI Search Index (with vector search support)
First, install the required Azure Search SDK (if not already):

In [28]:
!pip install azure-search-documents --quiet


In [14]:
!pip uninstall azure-search-documents -y
!pip install --upgrade pip setuptools

Found existing installation: azure-search-documents 11.6.0b11
Uninstalling azure-search-documents-11.6.0b11:
  Successfully uninstalled azure-search-documents-11.6.0b11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.0.0
    Uninstalling setuptools-68.0.0:
      Successfully uninstalled setuptools-68.0.0
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is

In [8]:
!pip install --upgrade azure-search-documents --pre


In [19]:
pip install azure-search-documents --pre --upgrade.


Usage:   
  /anaconda/envs/azureml_py38/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /anaconda/envs/azureml_py38/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /anaconda/envs/azureml_py38/bin/python -m pip install [options] [-e] <vcs project url> ...
  /anaconda/envs/azureml_py38/bin/python -m pip install [options] [-e] <local project path> ...
  /anaconda/envs/azureml_py38/bin/python -m pip install [options] <archive url/path> ...

no such option: --upgrade.
Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install --pre azure-search-documents==11.6.0b11


In [2]:
import azure.search.documents
print(azure.search.documents.__version__)


11.6.0b11


In [2]:
!pip install azure-search-documents azure-core

In [3]:
!pip install --pre --upgrade azure-search-documents


In [11]:
!pip install numpy


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/anaconda/envs/azureml_py38/lib/python3.10/site-packages/numpy-1.23.5.dist-info/METADATA'



In [10]:
!pip install --force-reinstall --no-deps numpy==1.23.5

  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
error: uninstall-no-record-file

× Cannot uninstall numpy 1.23.5
╰─> The package's contents are unknown: no RECORD file was found for numpy.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps numpy==1.23.5


In [12]:
import azure.search.documents
print("Version:", azure.search.documents.__version__)


Version: 11.6.0b11


In [2]:
from azureml.core import Environment

env = Environment.from_pip_requirements(name="fleet-vector-env", file_path="requirements.txt")
env.python.conda_dependencies.set_python_version("3.8")
env.register(workspace=ws)


NameError: name 'ws' is not defined

# Replace with your Azure AI Search details
search_service_name = "<YOUR_SEARCH_SERVICE_NAME>"  # e.g. fleet-search-service
admin_key = "<YOUR_SEARCH_ADMIN_KEY>"  # found in Azure portal under "Keys"
index_name = "fleet-maintenance-index"


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchableField,
    SimpleField,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    VectorSearchProfile
)


from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

# Replace with your Azure AI Search details
search_service_name = "fleet-search-service"
admin_key = ""
index_name = "fleet-maintenance-index"
endpoint = f"https://{search_service_name}.search.windows.net"

index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(admin_key))

# Define fields
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="filename", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(
        name="embedding",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="fleet-vector-profile"
    )
]

# Define vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswVectorSearchAlgorithmConfiguration(name="hnsw-config")
    ],
    profiles=[
        VectorSearchProfile(
            name="fleet-vector-profile",
            algorithm_configuration_name="hnsw-config"
        )
    ]
)

# Create index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)

# Delete existing index (if needed)
if index_name in [i.name for i in index_client.list_indexes()]:
    index_client.delete_index(index_name)
    print("Old index deleted.")

# Create the index
index_client.create_index(index)
print("✅ Index created successfully!")


ImportError: cannot import name 'HnswVectorSearchAlgorithmConfiguration' from 'azure.search.documents.indexes.models' (/anaconda/envs/azureml_py38/lib/python3.10/site-packages/azure/search/documents/indexes/models/__init__.py)